# Traveling Salesman Problem Algorithm using Genetic Algorithms

This notebook implements a genetic algorithm, that tries to "solve" the traveling salesman problem.

In [145]:
import random
import math
from IPython.core.debugger import set_trace
from IPython.display import clear_output

In [146]:
cities = []
numCities = 10

# The total size of the area, in which we will generate the cities
areaX = 20
areaY = 20

bestEver = []
recordDistance = 1e7
currentBest = []

population = []
numPopulation = 600
fitness = []

masterOrder = [x for x in range(numCities)]
# generate a random cities with position tuples of (x,y)
for i in range(numCities):
    newCity = (random.randint(1,areaX), random.randint(1,areaX))
    
    while newCity in cities:
        newCity = (random.randint(1,areaX), random.randint(1,areaY))
        
    cities.append(newCity)
    

In [147]:
# generate a population for the genetic algorithm
for i in range(numPopulation):
    order = random.sample(masterOrder, k=len(masterOrder))
    population.append(order)

In [148]:
# calculates the distance of a give order
def calcDistance(order):
    dist = 0
    for i in range(len(order)-1):
        #set_trace()
        dist += math.sqrt( (cities[order[i+1]][0] - cities[order[i]][0])**2 + (cities[order[i+1]][1] - cities[order[i]][1])**2 )
    return dist

In [149]:
# calculates the fitness values for the generated population
def calculateFitness():
    currentRecord = 1e7
    global recordDistance, bestEver
    for i in range(numPopulation):
        # calculate the distance of the orders in the population
        distance = calcDistance(population[i])
        
        # if the ditance is better than what we have ever seen before, save it
        if distance < recordDistance:
            recordDistance = distance
            bestEver = population[i]
        
        # if the ditance is better than what we have seen before in the population, save it
        if distance < currentRecord:
            currentRecord = distance
            currentBest = population[i]
            
        fitness.append( 1 / (pow(distance, 8) + 1))
        

In [150]:
# normalize the fitness
def normalizeFitness():
    theSUM = 0
    for i in range(len(fitness)):
        theSUM += fitness[i]
    for i in range(len(fitness)):
        fitness[i] = fitness[i] / theSUM

In [151]:
# pickes an element of a list by it's given probability
def pickOne(ls,prob):
    index = 0
    r = random.uniform(0,1)
    
    while r > 0:
        r = r - prob[index]
        index += 1
    
    index -= 1
    
    return ls[index]

In [152]:
# randomly mutates out order
def mutate(order, mutationRate):
    for i in range(numCities):
        if random.uniform(0,1) < mutationRate:
            index1 = math.floor(random.randint(0, len(order)-1))
            index2 = math.floor(random.randint(0, len(order)-1))
            #set_trace()
            order[index1], order[index2] = order[index2], order[index1]
            

In [153]:
def crossOver(order1, order2):
    start = math.floor(random.randint(0,len(order1)))
    end = math.floor(random.randint(start,len(order1)))
    neworder = order1[start:end]
    #set_trace()
    for i in range(len(order2)):
        city = order2[i]
        if city not in neworder:
            neworder.append(city)
    return neworder

In [154]:
def nextGeneration():
    newPopulation = []
    global population
    for i in range(numPopulation):
        order1 = pickOne(population, fitness)
        order2 = pickOne(population, fitness)
        order = crossOver(order1, order2)
        mutate(order,0.3)
        newPopulation.append(order)
    population = newPopulation

In [ ]:
import pygame
from math import pi
pygame.init()

screen = pygame.display.set_mode((500, 500))
WHITE = pygame.Color(255, 255, 255)
RED = pygame.Color(255, 0, 0) 
BLACK = pygame.Color(0, 0, 0) 

size = (500, 500)
radius = 5

canvas = pygame.Surface(size)

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            quit()
    
    calculateFitness()
    normalizeFitness()
    nextGeneration()
    print ("Current best order: " + str(bestEver) + " with a distance of " + str(recordDistance), end="\r")
    
    canvas.fill(BLACK)
    
    for city in cities:
        pygame.draw.circle(canvas, RED, (city[0]*20, city[1]*20), radius)

    for i in range(len(bestEver)-1):
        pygame.draw.line(canvas, WHITE, (cities[bestEver[i]][0]*20, cities[bestEver[i]][1]*20), (cities[bestEver[i+1]][0]*20, cities[bestEver[i+1]][1]*20))
    
    screen.blit(canvas,(0, 0))
    pygame.display.update()